In [42]:
import os
import pandas as pd
import openai
from dotenv import load_dotenv
import ast

In [43]:
df = pd.read_csv("subset_50_recipes.csv")

In [44]:
df.head

<bound method NDFrame.head of     Unnamed: 0                                              Title  \
0         7354                           Apple-Frangipane Galette   
1         1049                   Eggplant Omelet (Tortang Talong)   
2         6989                                    Blue Cheese Dip   
3          243                                             Hawaij   
4        12825           Spiced Fritters with Maple-Vanilla Syrup   
5         8749               Honey-Mustard Chicken-Sausage Kebabs   
6         7921               Maple Cake with Maple Syrup Frosting   
7        12259                              Turkey Cream Puff Pie   
8         9420  Eel With Olives, Chiles, and Capers (Anguilla ...   
9        10331                                     Pork Cassoulet   
10        3376          Thai Coconut, Broccoli and Coriander Soup   
11        5416                                Garlicky Mayonnaise   
12        7207                 Turkey Sweet Potato Shepherd's Pie   
13  

In [45]:
load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")

In [46]:
openai.api_key = API_KEY

def describe_dish(row):
    # Extract fields from the row
    title = row.get("Title", "Unknown Title")
    ingredients_raw = row.get("Cleaned_Ingredients", "")
    instructions = row.get("Instructions", "")

    # Attempt to parse the ingredients if they are stored as a string representation of a list
    try:
        ingredients_list = ast.literal_eval(ingredients_raw)
        if not isinstance(ingredients_list, list):
            ingredients_list = [str(ingredients_list)]
    except Exception:
        # Fallback: split by comma if parsing fails
        if isinstance(ingredients_raw, str):
            ingredients_list = [i.strip() for i in ingredients_raw.split(",")]
        else:
            ingredients_list = [str(ingredients_raw)]
    
    # Join the ingredients into a natural-sounding string
    ingredients_str = ", ".join(ingredients_list)
    
    # Summarize the instructions (if too long)
    instructions_summary = instructions if len(instructions) < 100 else instructions[:100] + "..."
    
    # Build a prompt for the AI that captures the vibe, flavor, and uniqueness of the dish
    prompt = f"""Write a unique and creative description for the following dish that emphasizes its flavor, texture, aroma, and overall vibe. The description should be engaging and make the dish sound appetizing.

Dish Title: {title}
Ingredients: {ingredients_str}
Preparation Overview: {instructions_summary}

Description:"""
    
    # Call OpenAI's completion endpoint to generate the description
    response = openai.Completion.create(
        model="text-davinci-003",  # You may switch to another model if needed.
        prompt=prompt,
        max_tokens=150,
        temperature=0.8,
        n=1,
        stop=None
    )
    
    # Extract and return the generated text
    description = response.choices[0].text.strip()
    return description


In [47]:
df["description"] = df.apply(describe_dish, axis=1)

APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
df.to_csv("recipes_with_descriptions.csv", index=False)